1) [x] Получите данные и загрузите их в рабочую среду. (Jupyter Notebook или другую)

In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv('heart.csv')

In [5]:
df.head(5)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


2) [x] Подготовьте датасет к обучению моделей:

a) Категориальные переменные переведите в цифровые значения. Можно использовать pd.get_dummies, preprocessing.LabelEncoder. Старайтесь не использовать для этой задачи циклы.

In [6]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()

In [7]:
X = df.copy().drop('HeartDisease', axis=1)
y = df['HeartDisease']

columns = [col for col, i in zip(X.columns, X.nunique()) if i < 50]
X = pd.get_dummies(X, columns=columns)

scaleX = ss.fit_transform(X)

3) [x] Разделите выборку на обучающее и тестовое подмножество. 80% данных оставить на обучающее множество, 20% на тестовое.

In [8]:
from sklearn.model_selection import train_test_split

In [9]:
X_train, X_test, y_train, y_test =  train_test_split(scaleX, y, test_size=.2, random_state=657)

4) [x] Обучите модель логистической регрессии с параметрами по умолчанию.

In [10]:
from sklearn.linear_model import LogisticRegression

In [11]:
LR_model = LogisticRegression(
    random_state=55
)
LR_model.fit(X_train, y_train)

LogisticRegression(random_state=55)

5) [x] Подсчитайте основные метрики модели. Используйте следующие метрики и функцию: cross_validate(…, cv=10, scoring=[‘accuracy’,‘recall’,‘precision’,‘f1’])

In [13]:
from sklearn.model_selection import cross_validate

In [145]:
def CV_scorer(model):
    CrosVal = cross_validate(model, X_train, y_train, cv=10, scoring=['accuracy', 'recall', 'precision', 'f1'])
    CrosVal = {'mean_'+key_ : val_.mean() for key_, val_ in CrosVal.items()}
    return CrosVal

In [146]:
CV_scorer(LR_model)

{'mean_fit_time': 0.0039866924285888675,
 'mean_score_time': 0.002598738670349121,
 'mean_test_accuracy': 0.8597556460570159,
 'mean_test_recall': 0.8895512820512821,
 'mean_test_precision': 0.8586887778067587,
 'mean_test_f1': 0.8731023169921018}

6) [x] Оптимизируйте 3-4 параметра модели:

- d) Повторите п. 5 после каждого итогового изменения параметров.

In [15]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

from sklearn.ensemble import StackingClassifier
from xgboost import XGBClassifier

<b>a) Используйте GridSearchCV.</b>

- Define search models

In [115]:
def make_cv_search(clf_model, estimator, params):
    if clf_model == GridSearchCV:
        clf = clf_model(estimator, params, n_jobs=3, cv=10)
    else:
        clf = clf_model(estimator, params, n_jobs=3, cv=10, n_iter=50)
    clf.fit(X_train, y_train)
    
    to_drop = [i for i in clf.cv_results_ if  not ('param' in i or 'test' in i)]
    
    return clf.best_score_, clf.best_estimator_, clf.best_params_

In [116]:
forest = RandomForestClassifier()

params = {
    'criterion' : ('gini', 'log_loss'),
    'min_samples_split' : [1, 5],
    'max_depth' : range(5, 11)
}

gsearchCV = make_cv_search(GridSearchCV, forest, params)

In [147]:
print(f'Best score: {gsearchCV[0]}')
print(f'Best estimator: {gsearchCV[1]}')

Best score: 0.8760644205849685
Best estimator: RandomForestClassifier(max_depth=9, min_samples_split=5)


In [248]:
cv1 = CV_scorer(gsearchCV[1])

<b>b) Используйте RandomizedSearchCV.</b>

- c) *Добавьте в п. 6b 2-5 моделей классификации и вариации их параметров.

In [182]:
models = [
    {'name' : 'RF', 'model': RandomForestClassifier(), 'params': {
        'n_estimators':[1, 5, 10, 15], 'criterion':['gini', 'entropy', 'log_loss'], 'max_depth': [5, 15, 25, 50], 'min_samples_split': [.005, .01, .02, .03, .04]
    }},
    {'name' : 'XGB', 'model': XGBClassifier(), 'params' : {
        'n_estimators':[1, 5, 10, 15], 'max_depth' : [1, 10, 25, 50], 'max_leaves': [0, 15, 25, 35, 55]
    }},
    {'name': 'KNN', 'model' : KNeighborsClassifier(), 'params' : {
        'n_neighbors' : [1, 5, 10, 25], 'leaf_size' : [1, 5, 10, 15], 'weights' : ['uniform', 'distance'], 'p':[1, 2],
    }}
]

In [183]:
estimators = []

for v in  models:
    cv_results = make_cv_search(RandomizedSearchCV, v['model'], v['params'])
    estimators.append(cv_results[1])
    print(v['name'], cv_results[0], cv_results[2])

RF 0.8706405035172159 {'n_estimators': 15, 'min_samples_split': 0.04, 'max_depth': 5, 'criterion': 'gini'}
XGB 0.8598111810440578 {'n_estimators': 10, 'max_leaves': 35, 'max_depth': 1}
KNN 0.877434283598667 {'weights': 'distance', 'p': 1, 'n_neighbors': 25, 'leaf_size': 1}


- Random Forest CV

In [184]:
cv2 = CV_scorer(estimators[0])

- XGB CV

In [185]:
cv3 = CV_scorer(estimators[1])

- KNN CV

In [186]:
cv4 = CV_scorer(estimators[2])

7) [ ] Сформулируйте выводы по проделанной работе:
- a) Сравните метрики построенных моделей.
- b) *Сравните с полученными результатами в домашнем задании по теме «Ансамблирование».

In [242]:
from sklearn.ensemble import StackingClassifier

In [245]:
stack_models = []

for name, est in zip(models, estimators):
    stack_models.append((name['name'], est))
    
stack = StackingClassifier(stack_models)
stack.fit(X_train, y_train)

stack_cv = CV_scorer(stack)

In [249]:
for k, val in cv1.items():
    cv1[k] = [val, cv2[k], cv3[k], cv4[k], stack_cv[k]]
    
res_df = pd.DataFrame().from_dict(cv1)
res_df.index = ['LR'] + [i['name'] for i in models] + ['Stack']

res_df

,mean_fit_time,mean_score_time,mean_test_accuracy,mean_test_recall,mean_test_precision,mean_test_f1
LR,0.176118,0.010767,0.870622,0.912244,0.859553,0.884035
RF,0.025722,0.005114,0.863847,0.917308,0.845784,0.879337
XGB,0.008211,0.004747,0.859811,0.919744,0.839264,0.876655
KNN,0.001222,0.033707,0.877434,0.914744,0.868106,0.889795
Stack,0.388286,0.038571,0.867901,0.909679,0.857309,0.881643


# Выводы:
1) Метрики моделей или хуже самой простой логистической регрессии или слегка лучше по различным показателям
2) Стэкинг оптимизированных моделей через рандомайз показал средние результаты :\

<b>Для решения проблемы недостаточного качества, предполагается, сделать следующее:</b>
 - Провести оптимизацию данных (feature engeneering)
 - Увеличить число итераций в рандомайз сёрче
 - Увеличить рэнджи значений параметров в рандомайз сёрче